# Notebook Base Entrenamiento

### Objetivo:
Transformación, formato y contrucción de la base de entrenamiento

Librerias

In [1]:
import pandas as pd
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import category_encoders as ce


**Lectura de datos**

In [3]:
 # get current directory
path = os.getcwd()

# parent directory
path = os.path.dirname(path)
path = path.replace('2_Model','1_Scraping')
data_scraping = pd.read_csv(path + "/datos_arriendos_consolidado.csv")
data_scraping.head()

,codigo,precio,zona,barrio_sector,tipo_pisos,baños_familiares,area_bruta,ciudad,numero_niveles,tipo_cocina,...,calentador_de_agua,terraza,closet_de_linos,biblioteca,parqueadero_visitantes,gimnasio,piscina,salon_social,dispositivos_automatizacion,alarma
0,Código 1306444 / Castilla,1100000.0,centro,castilla,ceramica,1.0,95.0,medellin zona 1 - centro,1.0,integral,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Código 1306376 / El Salvador,950000.0,centro,el salvador,ceramica,1.0,70.0,medellin zona 1 - centro,1.0,semi-integral,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Código 1306287 / Los Angeles,970000.0,centro,los angeles,ceramica,1.0,38.0,medellin zona 1 - centro,1.0,semi-integral,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Código 1306212 / Prado,1400000.0,centro,prado,ceramica,1.0,50.0,medellin zona 1 - centro,1.0,integral,...,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,Código 1306007 / 12 de Octubre,800000.0,centro,12 de octubre,ceramica,1.0,92.0,medellin zona 1 - centro,1.0,integralcon alacenas,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Se crean dos listas auxiliares para identificar las columnas numéricas de aquellas categóricas

In [3]:
# Lista de columnas numéricas
num_colums = ['precio',
              'baños_familiares',
              'area_bruta',
              'numero_niveles',
              'parqueaderos',
              'alcobas_familiares',
              'estrato',
              'area_total',
              'juegos_infantiles',
              'balcon', 
              'zona_ropas', 
              'camaras_cctv', 
              'cancha_polideportiva',
              'ascensor', 
              'cancha_squash', 
              'zona_bbq', 
              'patio',
              'unidad_cerrada_conjunto', 
              'zonas_verdes', 
              'aire_acondicionado',
              'jacuzzi', 
              'red_de_Gas', 
              'turco', 
              'porteria_24_7', 
              'sauna',
              'calentador_de_agua', 
              'terraza', 
              'closet_de_linos', 
              'biblioteca',
              'parqueadero_visitantes', 
              'gimnasio', 
              'piscina', 
              'salon_social',
              'dispositivos_automatizacion', 
              'alarma']

# Lista de columnas numéricas
cat_columns = ['zona', 
               'barrio_sector', 
               'tipo_pisos',
               'ciudad', 
               'tipo_cocina']

Se realiza un análisis de las columnas categóricas. Primero se visualiza la cantidad de categorísas que tiene cada una de estas y así definir la mejor estrategia de encoding con el fin de evitar problemas con la dimensión de la base

In [4]:
for cat in cat_columns:
    print(f"{cat}: ",len(data_scraping[cat].unique()))

zona:  5
barrio_sector:  170
tipo_pisos:  104
ciudad:  5
tipo_cocina:  75


Dado lo anterior se obtiene que las variables tipo_pisos, barrio_sector y tipo_cocina tienen demasiadas categorías para considerar hacer un encoding tradicional, por tanto, se debe realizar una estrategia de reducción de categorias y luego aplicar la estrategia de encoding que no aumente la base en su dimensión.

Adicional podemos concluir que la variable barrio_sector no será parte del modelo, por tanto, no será necesario hacer intervención

****

**Reducción de categorías:** 

Se procede a hacer reducción de categorías sobre las variables tipo_cocina y tipo_pisos. Esto consiste en consolidar aquellos valores similares en una sola categoria, ejm:

- ingral alacena <--> integral
- integral red de gas <--> integral

- madera laminada <--> madera
- madera laminada marmol <--> madera

In [5]:
# Se realiza la reducción de cateorías para la variable tipo cocina
df_concina = pd.read_csv('tipo_cocina.csv', sep=';')
dict_cocina = {}
for cocina in df_concina.itertuples():
    dict_cocina[cocina.original] = cocina.format

El archivo contiene en cada fila el valor original y su categoría asignada, esto se plica a cada valor en el data frame

In [6]:
data_scraping['tipo_cocina'] = data_scraping['tipo_cocina'].apply(lambda x: dict_cocina[x])

Se valida la reducción

In [7]:
data_scraping['tipo_cocina'].unique()

array(['integral', 'semiintegral', 'sencilla', 'otro', 'integralabierta',
       'mixta'], dtype=object)

Se valida la distribución de valores según la conversión

In [8]:
data_scraping.groupby('tipo_cocina').size()

tipo_cocina
integral           2786
integralabierta      25
mixta                25
otro                  8
semiintegral        263
sencilla            125
dtype: int64

Los datos anteriores muetran una gran diferencia en una de las categorías, esto puede implicar un aporte no muy diciente de esta variable en el modelo que se valida mas adelante

Se ejecuta el mismo proceso para la variable tipo_pisos

In [9]:
data_scraping['tipo_pisos'] = data_scraping['tipo_pisos'].apply(lambda x: x.replace('\n',''))

In [10]:
df_pisos = pd.read_csv('tipo_pisos.csv', sep=';')
dict_pisos = {}
for piso in df_pisos.itertuples():
    dict_pisos[piso.original] = piso.format

data_scraping['tipo_pisos'] = data_scraping['tipo_pisos'].apply(lambda x: dict_pisos[x])

In [11]:
data_scraping['tipo_pisos'].unique()

array(['ceramica', 'baldosa', 'granito', 'otro', 'marmol', 'porcelanato',
       'madera', 'ceramicamadera ', 'ceramicamadera', 'madera ',
       'porcelanamadera', 'marmolmadera', 'marmolceramica',
       'porcelanatomadera', 'ceramicaporcelanato', nan, 'tapetemadera',
       'ceramicagranito', 'ceramicabaldosa'], dtype=object)

In [12]:
data_scraping.groupby('tipo_pisos').size()

tipo_pisos
baldosa                 122
ceramica               1922
ceramicabaldosa           1
ceramicagranito           1
ceramicamadera           39
ceramicamadera            5
ceramicaporcelanato       2
granito                   2
madera                  313
madera                    1
marmol                  183
marmolceramica            5
marmolmadera             27
otro                     29
porcelanamadera           4
porcelanato             537
porcelanatomadera        27
tapetemadera              1
dtype: int64

Similar a la variable de tipo_cocina el resultante es una catgoría con mas cantidad de valores que las demás, mas adelante se determina si esto hace que esta variable no sea aportante para el modelo

**Limpieza de valores NaN y Null:**

Como parte del tratamiento de datos se realiza el conteo de valores NaN y Null para determinar si deben ser eliminados o se debe aplicar una estrategia de imputación

In [15]:
data_scraping.isnull().sum()

codigo                          0
precio                          0
zona                            0
barrio_sector                   0
tipo_pisos                     11
baños_familiares                0
area_bruta                      0
ciudad                          0
numero_niveles                  0
tipo_cocina                     0
parqueaderos                    0
alcobas_familiares              0
estrato                         0
area_total                      0
juegos_infantiles               0
balcon                          0
zona_ropas                      0
camaras_cctv                    0
cancha_polideportiva            0
ascensor                        0
cancha_squash                   0
zona_bbq                        0
patio                           0
unidad_cerrada_conjunto         0
zonas_verdes                    0
aire_acondicionado              0
jacuzzi                         0
red_de_Gas                      0
turco                           0
porteria_24_7 

In [17]:
data_scraping.isna().sum()

codigo                          0
precio                          0
zona                            0
barrio_sector                   0
tipo_pisos                     11
baños_familiares                0
area_bruta                      0
ciudad                          0
numero_niveles                  0
tipo_cocina                     0
parqueaderos                    0
alcobas_familiares              0
estrato                         0
area_total                      0
juegos_infantiles               0
balcon                          0
zona_ropas                      0
camaras_cctv                    0
cancha_polideportiva            0
ascensor                        0
cancha_squash                   0
zona_bbq                        0
patio                           0
unidad_cerrada_conjunto         0
zonas_verdes                    0
aire_acondicionado              0
jacuzzi                         0
red_de_Gas                      0
turco                           0
porteria_24_7 

Se tienen 11 valores nulos en la variable tipo pisos, por lo insignificante de este número con respecto a los datos en general se eliminan de la base

In [18]:
data_scraping = data_scraping.dropna()

**Tratamiento datos categóricos:**

Aunque se realizó un proceso de reducción de categorías para las variables tipo pisos y tipo cocina se tienen las siguientes cantidad de categorías resultantes:
- tipo_pisos: 18 categorías
- tipo_cocina: 6 categorías

Aun con estos valores, realizar un proceso de encoding causaria un aumento considerable en la dimensión de la base, por tanto se considera una estrategia que no aumente en cantidad de variables al aplicar encoding sobre estas variables.

La estrategia es selccionada es coificar las variables categoricas por medio del método de frecuencia. Este proceso consiste en reemplazar el valor de cada categoría por la frecuencia que esta representa en la base general, convirtiendo así la variable original en numérica

In [20]:
# Asignar el tipo de dato adecuado
data_scraping['tipo_cocina'] = data_scraping['tipo_cocina'].astype('category')
data_scraping['tipo_pisos'] = data_scraping['tipo_pisos'].astype('category')

C:\Users\user\AppData\Local\Temp\ipykernel_7040\3365246408.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_scraping['tipo_cocina'] = data_scraping['tipo_cocina'].astype('category')
C:\Users\user\AppData\Local\Temp\ipykernel_7040\3365246408.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_scraping['tipo_pisos'] = data_scraping['tipo_pisos'].astype('category')


In [21]:
data_scraping.head()

,codigo,precio,zona,barrio_sector,tipo_pisos,baños_familiares,area_bruta,ciudad,numero_niveles,tipo_cocina,...,calentador_de_agua,terraza,closet_de_linos,biblioteca,parqueadero_visitantes,gimnasio,piscina,salon_social,dispositivos_automatizacion,alarma
0,Código 1306444 / Castilla,1100000.0,centro,castilla,ceramica,1.0,95.0,medellin zona 1 - centro,1.0,integral,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Código 1306376 / El Salvador,950000.0,centro,el salvador,ceramica,1.0,70.0,medellin zona 1 - centro,1.0,semiintegral,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Código 1306287 / Los Angeles,970000.0,centro,los angeles,ceramica,1.0,38.0,medellin zona 1 - centro,1.0,semiintegral,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Código 1306212 / Prado,1400000.0,centro,prado,ceramica,1.0,50.0,medellin zona 1 - centro,1.0,integral,...,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,Código 1306007 / 12 de Octubre,800000.0,centro,12 de octubre,ceramica,1.0,92.0,medellin zona 1 - centro,1.0,integral,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Instancia del método de encoding y aplicación sobre cada variable

In [25]:
# Count Encoding - fit on training data, transform test data
encoder = ce.CountEncoder(normalize=True)
data_scraping["tipo_cocina_freq"] = encoder.fit_transform(data_scraping["tipo_cocina"])
data_scraping["tipo_pisos_freq"] = encoder.fit_transform(data_scraping["tipo_pisos"])

C:\Users\user\AppData\Local\Temp\ipykernel_7040\300108801.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_scraping["tipo_cocina_freq"] = encoder.fit_transform(data_scraping["tipo_cocina"])
C:\Users\user\AppData\Local\Temp\ipykernel_7040\300108801.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_scraping["tipo_pisos_freq"] = encoder.fit_transform(data_scraping["tipo_pisos"])


Se comparan los resultados de la variable original con su respectivo encoder

In [26]:
data_scraping.groupby('tipo_cocina').size()

tipo_cocina
integral           2775
integralabierta      25
mixta                25
otro                  8
semiintegral        263
sencilla            125
dtype: int64

In [27]:
data_scraping.groupby('tipo_cocina_freq').size()

tipo_cocina_freq
0.002484       8
0.007762      50
0.038808     125
0.081652     263
0.861534    2775
dtype: int64

In [28]:
data_scraping.groupby('tipo_pisos').size()

tipo_pisos
baldosa                 122
ceramica               1922
ceramicabaldosa           1
ceramicagranito           1
ceramicamadera           39
ceramicamadera            5
ceramicaporcelanato       2
granito                   2
madera                  313
madera                    1
marmol                  183
marmolceramica            5
marmolmadera             27
otro                     29
porcelanamadera           4
porcelanato             537
porcelanatomadera        27
tapetemadera              1
dtype: int64

In [29]:
data_scraping.groupby('tipo_pisos_freq').size()

tipo_pisos_freq
0.000310       4
0.000621       4
0.001242       4
0.001552      10
0.008382      54
0.009003      29
0.012108      39
0.037876     122
0.056815     183
0.097175     313
0.166718     537
0.596709    1922
dtype: int64

**Base de modelado:**

Habiendo realizado el anterior tratamiento de datos se procede a seleccionar las columnas, almacenar la base y dar paso a la fase de modelado

In [30]:
columns = ['precio','zona', 'barrio_sector', 'baños_familiares', 'area_bruta',
       'numero_niveles','parqueaderos', 'alcobas_familiares',
       'estrato', 'area_total', 'juegos_infantiles', 'balcon', 'zona_ropas',
       'camaras_cctv', 'cancha_polideportiva', 'ascensor', 'cancha_squash',
       'zona_bbq', 'patio', 'unidad_cerrada_conjunto', 'zonas_verdes',
       'aire_acondicionado', 'jacuzzi', 'red_de_Gas', 'turco', 'porteria_24_7',
       'sauna', 'calentador_de_agua', 'terraza', 'closet_de_linos',
       'biblioteca', 'parqueadero_visitantes', 'gimnasio', 'piscina',
       'salon_social', 'dispositivos_automatizacion', 'alarma',
       'tipo_cocina_freq', 'tipo_pisos_freq']

data_scraping = data_scraping[columns]
data_scraping.to_csv("base_modelado.csv", index=False)